<a href="https://colab.research.google.com/github/Delson182/EletivaII/blob/main/An%C3%A1lise_Financeira_e_Previs%C3%A3o_de_S%C3%A9ries_Temporais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import streamlit as st
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN, GRU, Dropout
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Configurações da página
st.set_page_config(page_title="Previsão de Preços", layout="wide")

# Título da aplicação
st.title('📈 Sistema de Previsão de Preços Financeiros')
st.markdown("""
Previsão de preços usando redes neurais recorrentes (LSTM, GRU, SimpleRNN) com indicadores técnicos.
""")

# Sidebar - Controles do usuário
st.sidebar.header('Configurações')
ticker = st.sidebar.selectbox('Selecione o ativo:', ['AAPL', 'BTC-USD', 'GOOG', 'MSFT', 'ETH-USD'])
model_type = st.sidebar.selectbox('Selecione o modelo:', ['LSTM', 'GRU', 'SimpleRNN'])
days_to_predict = st.sidebar.slider('Dias para prever:', 7, 30, 14)
look_back = st.sidebar.slider('Janela histórica (look_back):', 30, 90, 60)
epochs = st.sidebar.slider('Épocas de treinamento:', 10, 100, 50)

# Funções auxiliares (mesmas do seu código original)
def add_technical_indicators(df):
    df = df.copy()
    df['SMA_20'] = df['Close'].rolling(window=20, min_periods=1).mean()
    df['EMA_20'] = df['Close'].ewm(span=20, adjust=False, min_periods=1).mean()

    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14, min_periods=1).mean()
    avg_loss = loss.rolling(window=14, min_periods=1).mean()
    rs = avg_gain / avg_loss
    df['RSI_14'] = 100 - (100 / (1 + rs))

    exp1 = df['Close'].ewm(span=12, adjust=False, min_periods=1).mean()
    exp2 = df['Close'].ewm(span=26, adjust=False, min_periods=1).mean()
    df['MACD'] = exp1 - exp2
    df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False, min_periods=1).mean()

    return df.dropna()

def build_model(model_type, input_shape):
    model = Sequential()
    model.add(Dropout(0.2, input_shape=input_shape))

    if model_type == 'LSTM':
        model.add(LSTM(64))
    elif model_type == 'GRU':
        model.add(GRU(64))
    elif model_type == 'SimpleRNN':
        model.add(SimpleRNN(64))

    model.add(Dense(days_to_predict))
    model.compile(optimizer='adam', loss='mse')
    return model

# Carregar dados
@st.cache_data
def load_data(ticker):
    data = yf.download(ticker, period='1y')[['Close']]
    return data

# Pré-processamento
def prepare_data(data):
    data_with_indicators = add_technical_indicators(data)
    features = data_with_indicators[['Close', 'SMA_20', 'EMA_20', 'RSI_14', 'MACD', 'Signal_Line']].values
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(features)

    X, y = [], []
    for i in range(len(scaled) - look_back - days_to_predict + 1):
        X.append(scaled[i:i+look_back])
        y.append(scaled[i+look_back:i+look_back+days_to_predict, 0])  # Close price

    return np.array(X), np.array(y), scaler

# Interface principal
def main():
    # Carregar dados
    data_load_state = st.text('Carregando dados...')
    data = load_data(ticker)
    data_load_state.text('Dados carregados!')

    # Mostrar dados brutos
    if st.checkbox('Mostrar dados brutos'):
        st.subheader('Dados brutos')
        st.write(data.tail())

    # Gráfico dos dados históricos
    st.subheader(f'Série Temporal de {ticker}')
    st.line_chart(data['Close'])

    # Preparar dados
    X, y, scaler = prepare_data(data)

    # Dividir em treino e teste
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Construir e treinar modelo
    model = build_model(model_type, (look_back, 6))  # 6 features
    history = model.fit(X_train, y_train,
                       epochs=epochs,
                       batch_size=32,
                       validation_data=(X_test, y_test),
                       verbose=0)

    # Avaliação do modelo
    st.subheader('Desempenho do Modelo')
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)

    # Métricas
    train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))

    col1, col2 = st.columns(2)
    col1.metric("RMSE (Treino)", f"{train_rmse:.4f}")
    col2.metric("RMSE (Teste)", f"{test_rmse:.4f}")

    # Gráfico de perda
    fig, ax = plt.subplots()
    ax.plot(history.history['loss'], label='Treino')
    ax.plot(history.history['val_loss'], label='Teste')
    ax.set_title('Perda do Modelo')
    ax.set_ylabel('Perda')
    ax.set_xlabel('Época')
    ax.legend()
    st.pyplot(fig)

    # Previsão futura
    st.subheader(f'Previsão para os próximos {days_to_predict} dias')
    last_sequence = X[-1]
    future_prices = []
    current_sequence = last_sequence.copy()

    for _ in range(days_to_predict):
        pred = model.predict(current_sequence[np.newaxis, ...])[0][0]
        future_prices.append(pred)
        new_row = np.roll(current_sequence[-1], -1)
        new_row[0] = pred  # Atualiza o preço de fechamento
        current_sequence = np.roll(current_sequence, -1, axis=0)
        current_sequence[-1] = new_row

    # Reverter a normalização
    dummy = np.zeros((len(future_prices), 6))
    dummy[:, 0] = future_prices
    future_prices = scaler.inverse_transform(dummy)[:, 0]

    # Criar DataFrame com previsões
    future_dates = pd.date_range(start=data.index[-1], periods=days_to_predict+1, freq='B')[1:]
    forecast_df = pd.DataFrame({
        'Data': future_dates,
        'Previsão': future_prices
    }).set_index('Data')

    # Mostrar previsões
    st.line_chart(forecast_df)
    st.write(forecast_df.style.format({'Previsão': '${:.2f}'}))

    # Download das previsões
    csv = forecast_df.to_csv().encode('utf-8')
    st.download_button(
        label="Baixar previsões como CSV",
        data=csv,
        file_name=f'previsao_{ticker}.csv',
        mime='text/csv',
    )

if __name__ == '__main__':
    main()

2025-06-22 23:42:30.988 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:30.989 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:30.990 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:30.991 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:30.993 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:30.993 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:30.994 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:30.995 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


2025-06-22 23:42:46.653 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:46.653 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:46.654 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:46.655 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:46.656 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:46.657 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:46.658 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:46.659 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


2025-06-22 23:42:48.324 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:48.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:48.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:48.331 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:48.337 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:48.338 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:48.339 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 23:42:48.340 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar